In [1]:
print('e')

e


In [51]:
import re
import PyPDF2
import io
import google.generativeai as genaiI
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor
import os
import tempfile
import json
from langchain_community.document_loaders import PyPDFLoader
# from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [52]:
pdf_path = "C:\AI-Recruitment-agent\datascienceresume32 - Google Docs.pdf"

In [53]:
loder = PyPDFLoader(pdf_path)


In [54]:
documents = loder.load()

# print(documents[0].page_content)


In [55]:
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEYY = os.getenv("GEMINI_API")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)


In [56]:
result = llm.invoke('hii')
print(result.content)

Hi there! How can I help you today?



In [92]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEYY)
chunks = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in chunks]  # Extract text from Document objects
vectorstore = FAISS.from_texts(texts, embeddings)



In [117]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {input} 
""")
     



In [118]:
retriever = vectorstore.as_retriever()


In [119]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [120]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [121]:
response = retrieval_chain.invoke({"input": 'What is my fathers name?'})

In [122]:
print(response['answer'])

I'm sorry, but the provided text doesn't contain any information about your father's name.



In [123]:
print(response['context'][0].page_content)

●  Data Science Certification  -  Coursera  November 2023 
 ●  Earned Badge of Python and SQL  –  HackerRank  January 2024 
 ●  Five Star Python Coder -  HackerRank  May 2024 
 ADDITIONAL INFORMA TION 
 ●  Location :  Delhi 
 ●  Availability :  Immediate Joiner


In [124]:
skills_to_analyze = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" # Example of a skill that might not be explicitly mentioned
]


In [ ]:
skill_scores = []
for skill in skills_to_analyze:
    query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by resoning"
    response = retrieval_chain.invoke({"input": query})
    match = re.search(r'(\d+)', response['answer'])
    score = int(match.group(1) if match else 0)
    reasoning_parts = response['answer'].split('\n', 1)
    if len(reasoning_parts) > 1:
            
            reasoning = reasoning_parts[1].strip()
    else:
            reasoning = response['answer'].strip()
    # reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
    final_score = min(score, 10)
    skill_scores.append(final_score)
    print(f"  Score for {skill}: {final_score}/10")
    print(f"  Reasoning: {reasoning}")
# Overall score calculation
    
# overall_score = sum(skill_scores)
# n = len(skill_scores)
# print(f"Overall Score: {overall_score}/{n * 10}")
# find overll score
print(skill_scores)





  Score for Python: 9/10
  Reasoning: 
  Score for Java: 0/10
  Reasoning: 
  Score for React: 0/10
  Reasoning: 
  Score for PostgreSQL: 0/10
  Reasoning: 
  Score for Cloud Computing (AWS/Azure/GCP): 0/10
  Reasoning: 
  Score for Leadership: 0/10
  Reasoning: 
  Score for Project Management: 1/10
  Reasoning: Rating: 0
  Score for Machine Learning: 10/10
  Reasoning: 


In [ ]:
# final code
skill_scores = []
for skill in skills_to_analyze:
    query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by resoning"
    response = retrieval_chain.invoke({"input": query})
    match = re.search(r'(\d+)', response['answer'])
    score = int(match.group(1) if match else 0)
    reasoning_parts = response['answer']
    reasining_lines = [line.strip() for line in reasoning_parts.split('\n') if line.strip()]
    
    raw_reasoning = ' '.join(reasining_lines)
   
    final_resoning = re.sub(r'^\d+\s*[-.:]?\s*', '', raw_reasoning)
    # if len(reasoning_parts) > 1:
            
    #         reasoning = reasoning_parts[1].strip()
    # else:
    #         reasoning = response['answer'].strip()
    # reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
    final_score = min(score, 10)
    skill_scores.append(final_score)
    print(f"  Score for {skill}: {final_score}/10")
    print(f"  Reasoning: {final_resoning}")
# Overall score calculation
    
overall_score = sum(skill_scores)
n = len(skill_scores)
print(f"Overall Score: {overall_score}/{n * 10}")



  Score for Python: 9/10
  Reasoning: The candidate explicitly mentions Python under "Programming Languages" in the "TECHNICAL SKILLS" section and lists multiple projects and achievements highlighting their proficiency with the language (e.g., "Five Star Python Coder", developed multiple projects).
  Score for Java: 0/10
  Reasoning: Java is not mentioned.
  Score for React: 0/10
  Reasoning: The candidate does not mention React anywhere in the provided context.
  Score for PostgreSQL: 0/10
  Reasoning: The candidate doesn't mention PostgreSQL anywhere in the provided context.
  Score for Cloud Computing (AWS/Azure/GCP): 0/10
  Reasoning: There is no mention of Cloud Computing skills (AWS/Azure/GCP) in the provided context.
  Score for Leadership: 0/10
  Reasoning: There is no mention of Leadership in the provided context.
  Score for Project Management: 1/10
  Reasoning: Reasoning: The candidate doesn't explicitly mention or showcase proficiency in project management skills within the

In [130]:
match = re.search(r'(\d+)', response['answer'])
print(match)
score = int(match.group(1) if match else 0)
# print(score)
reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
print(reasoning)

<re.Match object; span=(0, 1), match='9'>



In [143]:
resoning_parts = response['answer'].strip().split('\n', 1)
if len(resoning_parts) > 1:
    reasoning = resoning_parts[1].strip()
else:   
    reasoning = response['answer'].strip()

print(reasoning)

9 - The candidate explicitly lists "Machine Learning" under the "Data Science" technical skills section, and provides project examples (Gemstone Price Predictor, Stock Price Forecasting using LSTM, MaternAI) that heavily rely on Machine Learning techniques. The experience at Infosys and Zidio Development also showcases the practical application of machine learning models.


In [144]:
# Ensure score is within 0-10 range
final_score = min(max(score, 0), 10)
print(f"  Score for {skill}: {final_score}/10")
print(f"  Reasoning: {reasoning}")

  Score for Machine Learning: 9/10
  Reasoning: 9 - The candidate explicitly lists "Machine Learning" under the "Data Science" technical skills section, and provides project examples (Gemstone Price Predictor, Stock Price Forecasting using LSTM, MaternAI) that heavily rely on Machine Learning techniques. The experience at Infosys and Zidio Development also showcases the practical application of machine learning models.


In [1]:
print('e')

e


In [ ]:
# import panda